In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout, Input, Attention
from tensorflow.keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
from keras.optimizers import Adam
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, concatenate,Flatten
from keras.regularizers import l2

2023-09-15 14:44:10.341114: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
train = pd.read_excel('/Vol1/sushant/Script/Assignment 1/ClassificationDataset-train2.xlsx')
val = pd.read_excel('/Vol1/sushant/Script/Assignment 1/ClassificationDataset-valid2.xlsx')

In [4]:
train.dropna(inplace=True)
val.dropna(inplace=True)

In [5]:
max_words = 10000
max_len = 100
embedding_dim = 200  # Correct the embedding dimension to match GloVe embeddings
num_classes = len(np.unique(train['ClassLabel']))
num_classes

5

In [6]:

le = LabelEncoder()
le.fit(train['ClassLabel'])

train_labels_encoded = le.transform(train['ClassLabel'])
val_labels_encoded = le.transform(val['ClassLabel'])
train_labels_onehot = np.eye(num_classes)[train_labels_encoded]
val_labels_onehot = np.eye(num_classes)[val_labels_encoded]


In [7]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['ReviewText'])
train_sequences = tokenizer.texts_to_sequences(train['ReviewText'])
val_sequences = tokenizer.texts_to_sequences(val['ReviewText'])
train_data_pad = pad_sequences(train_sequences, maxlen=max_len)
val_data_pad = pad_sequences(val_sequences, maxlen=max_len)


In [8]:
embedding_index = {}
with open('/Vol1/sushant/Script/Assignment 1/glove.twitter.27B.200d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coef = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coef

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [9]:
model = Sequential()
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_len, weights=[embedding_matrix], trainable=False)
input_layer = Input(shape=(max_len,))
embedded_sequences = embedding_layer(input_layer)
# Word-level Bi-LSTM
word_lstm = Bidirectional(LSTM(128, return_sequences=True))(embedded_sequences)
# Word-level attention
word_attention = Attention()([word_lstm, word_lstm])
# Sentence-level Bi-LSTM
sentence_lstm = Bidirectional(LSTM(128, return_sequences=False))(word_attention)
# Sentence-level attention
sentence_attention = Attention()([sentence_lstm, sentence_lstm])
# Document classifier
output_layer = Dense(num_classes, activation='softmax')(sentence_attention)
# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 100, 200)     2000000     ['input_1[0][0]']                
                                                                                                  
 bidirectional (Bidirectional)  (None, 100, 256)     336896      ['embedding[0][0]']              
                                                                                                  
 attention (Attention)          (None, 100, 256)     0           ['bidirectional[0][0]',          
                                                                  'bidirectional[0][0]']      

In [9]:
adam_optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [10]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)


In [11]:
model.fit(train_data_pad, train_labels_onehot, epochs=50, batch_size=64,
          validation_data=(val_data_pad, val_labels_onehot),
           callbacks=[early_stopping])

Epoch 1/50


2023-09-09 19:22:44.733785: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "111" frequency: 2400 num_cores: 40 environment { key: "cpu_instruction_set" value: "SSE, SSE2, SSE3" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 26214400 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


44/44 [==============================] - ETA: 0s - loss: 0.7566 - accuracy: 0.8016

2023-09-09 19:22:57.361689: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "111" frequency: 2400 num_cores: 40 environment { key: "cpu_instruction_set" value: "SSE, SSE2, SSE3" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 26214400 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


44/44 [==============================] - 19s 314ms/step - loss: 0.7566 - accuracy: 0.8016 - val_loss: 0.7343 - val_accuracy: 0.8002
Epoch 2/50
44/44 [==============================] - 17s 392ms/step - loss: 0.7324 - accuracy: 0.8016 - val_loss: 0.7338 - val_accuracy: 0.8002
Epoch 3/50
44/44 [==============================] - 17s 393ms/step - loss: 0.7292 - accuracy: 0.8016 - val_loss: 0.7348 - val_accuracy: 0.8002
Epoch 4/50
44/44 [==============================] - 16s 366ms/step - loss: 0.7284 - accuracy: 0.8016 - val_loss: 0.7304 - val_accuracy: 0.8002
Epoch 5/50
44/44 [==============================] - 17s 380ms/step - loss: 0.7283 - accuracy: 0.8016 - val_loss: 0.7315 - val_accuracy: 0.8002
Epoch 6/50
44/44 [==============================] - 17s 380ms/step - loss: 0.7275 - accuracy: 0.8016 - val_loss: 0.7333 - val_accuracy: 0.8002
Epoch 7/50
44/44 [==============================] - 16s 374ms/step - loss: 0.7319 - accuracy: 0.8016 - val_loss: 0.7394 - val_accuracy: 0.8002
Epoch 8/50

In [12]:
val_predictions = model.predict(val_data_pad)
val_predictions_classes = np.argmax(val_predictions, axis=1)
f1_scores = f1_score(val_labels_encoded, val_predictions_classes, average='micro')
f1_scores

2023-09-09 19:29:49.703318: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "111" frequency: 2400 num_cores: 40 environment { key: "cpu_instruction_set" value: "SSE, SSE2, SSE3" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 26214400 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


38/38 [==============================] - 5s 87ms/step


0.8118234804329725

In [13]:
model.save('dataset_2.h5')